# 02: Running the Agent

In Notebook 01 we explored the dataset and tools. This notebook shows how to run the
`KnowledgeGroundedAgent` in practice.

## What You'll Learn

1. The agent's PlanReAct architecture and the `AgentResponse` data structure
2. Running a question with live progress display
3. Inspecting the response: plan, tool calls, sources, and reasoning
4. Multi-turn conversations using session state
5. Observability with Langfuse tracing

## Prerequisites

Complete Notebook 01. You'll need `GOOGLE_API_KEY` in your `.env` file.
For tracing (Section 4): `LANGFUSE_PUBLIC_KEY` and `LANGFUSE_SECRET_KEY` are also required.

In [ ]:
import os
import uuid
from pathlib import Path

from aieng.agent_evals.knowledge_qa import KnowledgeGroundedAgent
from aieng.agent_evals.knowledge_qa.notebook import display_response, run_with_display
from aieng.agent_evals.langfuse import init_tracing
from dotenv import load_dotenv
from rich.console import Console
from rich.panel import Panel
from rich.table import Table


if Path("").absolute().name == "eval-agents":
    print(f"Working directory: {Path('').absolute()}")
else:
    os.chdir(Path("").absolute().parent.parent)
    print(f"Working directory set to: {Path('').absolute()}")

load_dotenv(verbose=True)
console = Console(width=100)

## 1. Agent Architecture

The `KnowledgeGroundedAgent` is built on Google ADK and combines two patterns:

**PlanReAct** — Before executing, the agent produces an explicit research plan with numbered
steps. Each step has a type (`SEARCH`, `FETCH`, `ANALYZE`) and a status that transitions from
`pending` → `in_progress` → `completed` (or `failed`/`skipped`). The plan can be revised
mid-run if the agent encounters unexpected results.

**ReAct loop** — Within each step, the agent alternates between reasoning (Thought), acting
(tool call), and observing (tool response).

### The `AgentResponse` Object

After running, `agent.answer_async(question)` returns an `AgentResponse`:

| Field | Type | Description |
|-------|------|-------------|
| `text` | `str` | The final answer |
| `plan` | `ResearchPlan` | Numbered steps with statuses |
| `tool_calls` | `list[dict]` | Every tool invocation during execution |
| `sources` | `list[GroundingChunk]` | URLs used as evidence |
| `reasoning_chain` | `list[str]` | The model's intermediate reasoning |
| `total_duration_ms` | `int` | Wall-clock execution time |

In [ ]:
agent = KnowledgeGroundedAgent(enable_planning=True)

config_table = Table(title="Agent Configuration", show_header=False)
config_table.add_column("Setting", style="cyan")
config_table.add_column("Value", style="white")
config_table.add_row("Model", agent.model)
config_table.add_row("Planning", "PlanReAct (enabled)")
config_table.add_row("Session Service", "InMemorySessionService")
config_table.add_row("Tools", "google_search, web_fetch, fetch_file, grep_file, read_file")

console.print(config_table)

## 2. Running a Question

`run_with_display` executes the agent in a Jupyter notebook with a live progress display showing:

- The research plan with step statuses (updating in real time)
- Tool calls as they fire

We'll use a question that requires web search — the agent must find and verify a specific fact,
not recall it from training data.

In [ ]:
question = "When was the highest single-day snowfall recorded in Toronto, and how much snow fell?"

response = await run_with_display(agent, question)

In [ ]:
display_response(
    console,
    response.text,
    subtitle=(
        f"Duration: {response.total_duration_ms / 1000:.1f}s  |  "
        f"Tool calls: {len(response.tool_calls)}  |  "
        f"Sources: {len(response.sources)}"
    ),
)

### 2.1 Inspecting the Response

The `AgentResponse` object contains the full execution trace.

In [ ]:
plan = response.plan

plan_table = Table(title="Research Plan")
plan_table.add_column("#", style="cyan", width=3)
plan_table.add_column("Step", style="white")
plan_table.add_column("Type", style="dim", width=12)
plan_table.add_column("Status", style="green")

for step in plan.steps:
    icon = {"completed": "✓", "failed": "✗", "skipped": "○"}.get(step.status.value, "·")
    desc = step.description[:70] + "..." if len(step.description) > 70 else step.description
    plan_table.add_row(str(step.step_id), desc, step.step_type, f"{icon} {step.status.value}")

console.print(plan_table)

In [ ]:
if response.tool_calls:
    tools_table = Table(title="Tool Calls")
    tools_table.add_column("#", style="dim", width=3)
    tools_table.add_column("Tool", style="cyan", width=16)
    tools_table.add_column("Arguments (truncated)", style="white")

    for i, tc in enumerate(response.tool_calls[:15], 1):
        name = tc.get("name", "unknown")
        args = str(tc.get("args", {}))
        args = args[:70] + "..." if len(args) > 70 else args
        tools_table.add_row(str(i), name, args)

    if len(response.tool_calls) > 15:
        tools_table.add_row("...", f"({len(response.tool_calls) - 15} more)", "")

    console.print(tools_table)
else:
    console.print("[dim]No tool calls recorded[/dim]")

In [ ]:
if response.sources:
    seen: set[str] = set()
    sources_table = Table(title="Sources")
    sources_table.add_column("#", style="dim", width=3)
    sources_table.add_column("URL", style="blue")

    for src in response.sources:
        if src.uri and src.uri not in seen:
            seen.add(src.uri)
            url = src.uri[:85] + "..." if len(src.uri) > 85 else src.uri
            sources_table.add_row(str(len(seen)), url)
        if len(seen) >= 10:
            break

    console.print(sources_table)
else:
    console.print("[dim]No sources recorded[/dim]")

## 3. Multi-Turn Conversations

The agent uses an `InMemorySessionService` to maintain conversation context across turns.
Pass the same `session_id` to link questions together — the agent will use prior context
when answering follow-up questions.

In [ ]:
session_id = str(uuid.uuid4())
console.print(f"Session ID: [dim]{session_id}[/dim]\n")

# First turn: establish a subject
response1 = await agent.answer_async(
    "What is the capital of France?",
    session_id=session_id,
)
display_response(console, response1.text, title="Turn 1")

# Second turn: follow-up that references the prior context
response2 = await agent.answer_async(
    "What is the official language spoken there?",
    session_id=session_id,
)
display_response(console, response2.text, title="Turn 2 (follow-up)")

## 4. Observability with Langfuse

Langfuse captures a full trace of every agent run using OpenTelemetry, giving you visibility into:

- Every tool call and its arguments
- Every LLM call with prompts and completions
- Timing for each span
- The full agent execution tree

This is essential for debugging failures, measuring latency, and comparing configurations.

### Trace Structure

```
Trace: agent run
├── Span: planning (PlanReAct)
│   └── LLM Call: create_plan
├── Span: step-1-execution
│   ├── Tool Call: google_search
│   ├── Tool Call: web_fetch
│   └── LLM Call: step_summary
├── Span: step-2-execution
│   └── ...
└── Span: synthesis
    └── LLM Call: final_answer
```

### Prerequisites

Set these in your `.env` file:
- `LANGFUSE_PUBLIC_KEY`
- `LANGFUSE_SECRET_KEY`
- `LANGFUSE_HOST` (optional, defaults to `https://cloud.langfuse.com`)

In [ ]:
langfuse_configured = all(
    [
        os.getenv("LANGFUSE_PUBLIC_KEY"),
        os.getenv("LANGFUSE_SECRET_KEY"),
    ]
)

if langfuse_configured:
    console.print("[green]✓[/green] Langfuse credentials found")
else:
    console.print("[yellow]⚠[/yellow] Langfuse credentials not found — tracing cells will be skipped")
    console.print("[dim]Set LANGFUSE_PUBLIC_KEY and LANGFUSE_SECRET_KEY in .env[/dim]")

In [ ]:
tracing_enabled = init_tracing()

if tracing_enabled:
    console.print("[green]✓[/green] Langfuse tracing initialized")
else:
    console.print("[yellow]⚠[/yellow] Tracing not enabled (check credentials)")

In [ ]:
if tracing_enabled:
    from langfuse import Langfuse

    langfuse = Langfuse()
    traced_agent = KnowledgeGroundedAgent(enable_planning=True)
    traced_question = "What programming language was created by Guido van Rossum, and in what year?"

    console.print(Panel(traced_question, title="Traced Question", border_style="green"))

    with langfuse.start_as_current_span(name="knowledge-agent", input=traced_question):
        trace_id = langfuse.get_current_trace_id()
        traced_response = await traced_agent.answer_async(traced_question)
        langfuse.update_current_span(output=traced_response.text)

    display_response(
        console,
        traced_response.text,
        subtitle=f"Duration: {traced_response.total_duration_ms / 1000:.1f}s",
    )
else:
    console.print("[dim]Skipping (Langfuse not configured)[/dim]")
    trace_id = None

In [ ]:
if tracing_enabled:
    from IPython.display import HTML, display  # noqa: A004
    from langfuse import Langfuse
    from opentelemetry import trace as otel_trace

    provider = otel_trace.get_tracer_provider()
    if hasattr(provider, "force_flush"):
        provider.force_flush(timeout_millis=5000)
    console.print("[green]✓[/green] Traces flushed to Langfuse")

    if trace_id:
        trace_url = Langfuse().get_trace_url(trace_id=trace_id)
        display(HTML(f'<p>View trace: <a href="{trace_url}" target="_blank">{trace_url}</a></p>'))

### 4.1 Viewing Traces in the Langfuse UI

Open your Langfuse project and navigate to **Traces**. Each run appears as a
tree of spans. Useful things to look at:

- **Span timeline** — which steps take the most time?
- **Tool call arguments** — what search queries did the agent use?
- **LLM interactions** — what did the model reason about before calling each tool?
- **Errors** — red spans show where failures occurred

You can also filter by trace name, time range, or input content.

## Summary

In this notebook you learned:

1. **Creating the agent** — `KnowledgeGroundedAgent(enable_planning=True)` with PlanReAct
2. **Running questions** — `run_with_display` for live notebook progress; `agent.answer_async` for raw access
3. **The `AgentResponse`** — plan, tool calls, sources, reasoning, and timing in one object
4. **Multi-turn conversations** — linking turns with `session_id`
5. **Langfuse tracing** — `init_tracing()` and the Langfuse SDK for full observability

**Next:** In Notebook 03, we'll run a systematic evaluation using the DeepSearchQA benchmark.

In [ ]:
console.print(
    Panel(
        "[green]✓[/green] Notebook complete!\n\n"
        "[cyan]Next:[/cyan] Open [bold]03_evaluation.ipynb[/bold] to evaluate the agent at scale.",
        title="Done",
        border_style="green",
    )
)